In [15]:
import pandas as pd 
import os
import json

from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

import torch
import torch.nn.functional as F

import seaborn as sns

from datasets import load_dataset, load_from_disk
import dtale

from datasets import load_dataset

from collections import Counter

import ast

In [4]:
deltabench_dataset = pd.DataFrame(load_dataset("OpenStellarTeam/DeltaBench")['train'])

Repo card metadata block was not found. Setting CardData to empty.


In [7]:
deltabench_dataset

,id,origin,task_l1,task_l2,question,solution,answer,long_cot,sections,sections_content,sections_labeled_info,final_correct,stragegy_shift_section_numbers,reason_unuseful_section_numbers,reason_error_section_numbers,reflection_section_numbers
0,f11c7a6bbf5754fee116b638d99a33a6,Omni-MATH_pascal,math,Number Theory,"We call the pair $(m, n)$ of positive integers...","Suppose that $(205800, 35k)$ is a happy pair. ...",30,Let's tackle this problem step by step. we nee...,"[{'start': 1, 'end': 3, 'description': 'Introd...",section1: Let's tackle this problem step by st...,"[{'section_number': 1, 'strategy_shift': '0', ...",0,[2],[],[7],[]
1,676adb9206bdd7cd09a9a20abec112da,Omni-MATH_HMMT_11,math,Number Theory,Call a number feared if it contains the digits...,"First of all, note that we cannot have $n, n+a...",1287,Let's tackle this problem step by step. it's a...,"[{'start': 1, 'end': 2, 'description': 'Define...",section1: Let's tackle this problem step by st...,"[{'section_number': 1, 'strategy_shift': '0', ...",0,"[3, 5, 6]",[],[7],[]
2,17af0b1210d442d79a4a9a512102eab0,Omni-MATH_HMMT_11,math,Number Theory,A number $n$ is $b a d$ if there exists some i...,Call a number good if it is not bad. We claim ...,25,"So i'm trying to solve this problem about ""bad...","[{'start': 1, 'end': 5, 'description': 'Introd...",section1: So i'm trying to solve this problem ...,"[{'section_number': 1, 'strategy_shift': '0', ...",0,[2],[],[2],[]
3,15b7ef6c3f5ca4ec9900c20e585c7e15,Omni-MATH_HMMT_2,math,Number Theory,Our next object up for bid is an arithmetic pr...,12. We can get 12 with 110437 and difference 1...,12,"Alright, so I've got this problem here: I need...","[{'start': 1, 'end': 3, 'description': 'Introd...","section1: Alright, so I've got this problem he...","[{'section_number': 1, 'strategy_shift': '0', ...",0,"[3, 5, 6, 11, 12, 13, 15, 17, 19]",[],[14],"[8, 12, 14, 18]"
4,d5bbdb2d68e9123f5953cd893bdbc7cc,Omni-MATH_HMMT_2,math,Number Theory,Estimate the number of positive integers $n \l...,Let $N$ denote $10^{6}$. We count by summing o...,757575,So i've got this problem: i need to estimate t...,"[{'start': 1, 'end': 2, 'description': 'Introd...",section1: So i've got this problem: i need to ...,"[{'section_number': 1, 'strategy_shift': '1', ...",0,"[1, 3, 6, 7, 9, 10, 12, 14, 16]",[],[16],"[5, 6, 8]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,d25fad8b726fe455164951e06ee629a9,MatrixStudio/Codeforces-Python-Submissions,code,Others,Title: Trains\n\nTime Limit: 2 seconds\n\nMemo...,```python\n# LUOGU_RID: 104833737\nfrom math i...,None,Let's analyze the input and the problem statem...,"[{'start': 1, 'end': 4, 'description': 'Analyz...",section1: Let's analyze the input and the prob...,"[{'section_number': 1, 'strategy_shift': '0', ...",0,[],[10],"[4, 9]",[]
1232,531e2b9acfc4e1a1f8824407fbca5468,Administrative Aptitude Test,general,Logical Reasoning,"In a dormitory, there are four people: A, B, C...",B \n 解析:<p>第一步，确定题型。<br/>题干有信息匹配特征，确定为分析推理。<br...,None,"Let's try to figure out who among A, B, C, and...","[{'start': 1, 'end': 6, 'description': 'Proble...","section1: Let's try to figure out who among A,...","[{'section_number': 1, 'strategy_shift': '0', ...",0,[],[],"[8, 4, 6, 7]","[4, 6, 7]"
1233,7f2b7da7f69f770f9cc462ee3a6a4fe0,https://artofproblemsolving.com/wiki/index.php...,math,Algebra,Let \$x_1\leq x_2\leq \cdots\leq x_{100}\$ be ...,To find the greatest value of \$x_{76} - x_{16...,841,The problem asks for the maximum value of the ...,"[{'start': 1, 'end': 3, 'description': 'Proble...",section1: The problem asks for the maximum val...,"[{'section_number': 1, 'strategy_shift': '0', ...",0,[2],[],[9],[]
1234,3652a958890063cfc22471708f03df9a,MatrixStudio/Codeforces-Python-Submissions,code,Graph Algorithms,Title: Arpa's loud Owf and Mehrdad's evil plan...,"```python\ndef gcd(a, b):\n return a if b =...",None,Let's analyze the input and the problem statem...,"[{'start': 1, 'end': 3, 'description': 'Introd...",se

In [33]:
example = deltabench_dataset.iloc[20]

In [38]:
sections = ast.literal_eval(example["sections"])
section_content = example["sections_content"]
section_labels = ast.literal_eval(example["sections_labeled_info"])
error_section = ast.literal_eval(example["reason_error_section_numbers"])

In [39]:
sections

[{'start': 1,
  'end': 2,
  'description': 'Problem statement and breakdown of the scenario',
  'content': "Alright, so I've got this probability problem here. Bob and Alice each have a bag with one ball of each color: blue, green, orange, red, and violet. So, each bag has exactly five balls, one of each color.\n\nFirst, Alice randomly selects one ball from her bag and puts it into Bob's bag. Then, Bob randomly selects one ball from his bag and puts it into Alice's bag. After this exchange, we need to find the probability that both bags have the same contents.",
  'section_type': 'Problem Analysis'},
 {'start': 3,
  'end': 8,
  'description': 'Initial analysis of the problem and understanding the conditions for the bags to have the same contents',
  'content': "Okay, let's break this down step by step. Initially, both bags have the same set of balls: blue, green, orange, red, and violet. So, each has one of each color.\n\nStep 1: Alice picks one ball from her bag and puts it into Bob's

In [36]:
error_section

[2, 3, 7, 8]

In [26]:
example

id                                                  3652a958890063cfc22471708f03df9a
origin                                    MatrixStudio/Codeforces-Python-Submissions
task_l1                                                                         code
task_l2                                                             Graph Algorithms
question                           Title: Arpa's loud Owf and Mehrdad's evil plan...
solution                           ```python\ndef gcd(a, b):\n    return a if b =...
answer                                                                          None
long_cot                           Let's analyze the input and the problem statem...
sections                           [{'start': 1, 'end': 3, 'description': 'Introd...
sections_content                   section1: Let's analyze the input and the prob...
sections_labeled_info              [{'section_number': 1, 'strategy_shift': '0', ...
final_correct                                                    